### a. 共計有多少Twitter參與這類關鍵字議題討論？106

In [59]:
from elasticsearch import Elasticsearch
import json

client = Elasticsearch('localhost:9200')

response = client.search(
    index = "twitter2",
    body = {
        "size" : 0,
        "_source": {
            "exclude": [ "user.*", "retweeted_status.*" ]
        },
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": {
                                "type": "boolean",
                                "query": "Reversing Vulnerability CVE exploitation Spoofing password DDoS SSL CryptoLocker Encrypt",
                                "operator": "or"
                            }
                        }
                    },
                    {
                        "range": {
                            "created_at": {
                                "gte": "Sun Jan 01 00:00:00 +0000 2012",
                                "lt": "Thu Jan 01 00:00:00 +0000 2015"
                            }
                        }
                    }
                ]
            }
        },
        "aggs": {
            "twitter_accounts_in_range": {
                "cardinality" : {
                    "field" : "uid"
                }
            }
        }
    }
)

# print (json.dumps(response, indent = 2))
print (response["aggregations"]["twitter_accounts_in_range"]["value"])

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.022s]


106


### b. 共計有多少Tweets? 4945

In [52]:
from elasticsearch import Elasticsearch

client = Elasticsearch('localhost:9200')

response = client.search(
    index = "twitter2",
    body = {
        "size" : 0,
        "_source": {
            "exclude": [ "user.*", "retweeted_status.*" ]
        },
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": {
                                "type": "boolean",
                                "query": "Reversing Vulnerability CVE exploitation Spoofing password DDoS SSL CryptoLocker Encrypt",
                                "operator": "or"
                            }
                        }
                    },
                    {
                        "range": {
                            "created_at": {
                                "gte": "Sun Jan 01 00:00:00 +0000 2012",
                                "lt": "Thu Jan 01 00:00:00 +0000 2015"
                            }
                        }
                    }
                ]
            }
        }
    }
)

# print (json.dumps(response, indent = 2))
print (json.dumps(response["hits"]["total"]))

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.027s]


4945


### c. 每個月參與的Twitter數量長條圖

In [53]:
from elasticsearch import Elasticsearch
from bokeh.charts import Bar, reset_output, output_notebook, show
# from bokeh.io import output_notebook
import json

client = Elasticsearch('localhost:9200')

response = client.search(
    index = "twitter2",
    body = {
        "size" : 0,
        "_source": {
            "exclude": [ "user.*", "retweeted_status.*" ]
        },
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": {
                                "type": "boolean",
                                "query": "Reversing Vulnerability CVE exploitation Spoofing password DDoS SSL CryptoLocker Encrypt",
                                "operator": "or"
                            }
                        }
                    },
                    {
                        "range": {
                            "created_at": {
                                "gte": "Sun Jan 01 00:00:00 +0000 2012",
                                "lt": "Thu Jan 01 00:00:00 +0000 2015"
                            }
                        }
                    }
                ]
            }
        },
        "aggs": {
            "tweets_by_month": {
                "date_histogram": {
                    "field": "created_at",
                    "interval": "month",
                    "format": "yyyy-MM",
                    "min_doc_count" : 0
                },
                "aggs" : {
                    "users_per_month" : {
                        "cardinality" : {
                            "field" : "uid"
                        }
                    }
                }
            }
        }
    }
)


data = dict(month=[], users_per_month=[])

for d in response["aggregations"]["tweets_by_month"]["buckets"]:
    data["month"].append(d["key_as_string"])
    data["users_per_month"].append(d["users_per_month"]["value"])
# print (data)
# bar = Bar(data, label="month", values="users_per_month")
reset_output()
output_notebook()
show(Bar(data, label="month", values="users_per_month", legend=False, color="black", plot_width=800), notebook_handle=True)
# print (json.dumps(response, indent = 2))
# print (response["aggregations"]["users"]["value"])

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.043s]


Loading BokehJS ...

### d. 每個月參Tweets數量長條圖

In [54]:
from elasticsearch import Elasticsearch
from bokeh.charts import Bar, reset_output, output_notebook, show
# from bokeh.io import output_notebook
import json

client = Elasticsearch('localhost:9200')

# search_type=count means return 
response = client.search(
    index = "twitter2",
    body = {
        "size" : 0,
        "_source": {
            "exclude": [ "user.*", "retweeted_status.*" ]
        },
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": {
                                "type": "boolean",
                                "query": "Reversing Vulnerability CVE exploitation Spoofing password DDoS SSL CryptoLocker Encrypt",
                                "operator": "or"
                            }
                        }
                    },
                    {
                        "range": {
                            "created_at": {
                                "gte": "Sun Jan 01 00:00:00 +0000 2012",
                                "lt": "Thu Jan 01 00:00:00 +0000 2015"
                            }
                        }
                    }
                ]
            }
        },
        "aggs": {
            "tweets_by_month": {
                "date_histogram": {
                    "field": "created_at",
                    "interval": "month",
                    "format": "yyyy-MM",
                    "min_doc_count" : 0
                }
            }
        }
    }
)

data = dict(month=[], tweets_per_month=[])
for d in response["aggregations"]["tweets_by_month"]["buckets"]:
    data["month"].append(d["key_as_string"])
    data["tweets_per_month"].append(d["doc_count"])

reset_output()
output_notebook()
show(Bar(data, label="month", values="tweets_per_month", legend=False, color="black", plot_width=800),
     notebook_handle=True)
# print ( json.dumps(response, indent = 2) )
# print (response["aggregations"]["users"]["value"])

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.019s]


Loading BokehJS ...

### e. 其中有URL與無URL的Tweets的比例圓餅圖

In [55]:
from elasticsearch import Elasticsearch
from bokeh.charts import Donut, reset_output, output_notebook, show
# from bokeh.io import output_notebook
import json, math

client = Elasticsearch('localhost:9200')

# search_type=count means return 
total_response = client.search(
    index = "twitter2",
    body = {
        "size" : 0,
        "_source": {
            "exclude": [ "user.*", "retweeted_status.*" ]
        },
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": {
                                "type": "boolean",
                                "query": "Reversing Vulnerability CVE exploitation Spoofing password DDoS SSL CryptoLocker Encrypt",
                                "operator": "or"
                            }
                        }
                    },
                    {
                        "range": {
                            "created_at": {
                                "gte": "Sun Jan 01 00:00:00 +0000 2012",
                                "lt": "Thu Jan 01 00:00:00 +0000 2015"
                            }
                        }
                    }
                ]
            }
        }
    }
)

no_url_response = client.search(
    index = "twitter2",
    body = {
        "size" : 0,
        "_source": {
            "exclude": [ "user.*", "retweeted_status.*" ]
        },
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": {
                                "type": "boolean",
                                "query": "Reversing Vulnerability CVE exploitation Spoofing password DDoS SSL CryptoLocker Encrypt",
                                "operator": "or"
                            }
                        }
                    },
                    {
                        "range": {
                            "created_at": {
                                "gte": "Sun Jan 01 00:00:00 +0000 2012",
                                "lt": "Thu Jan 01 00:00:00 +0000 2015"
                            }
                        }
                    }
                ]
            }
        },
        "filter": {
            "bool": {
                "must": [
                    {
                        "missing": {
                            "field": "entities.urls"
                        }
                    }
                ]
            }
        }
    }
)

totalData = total_response["hits"]["total"]
withUrl = totalData - no_url_response["hits"]["total"]
withoutUrl = no_url_response["hits"]["total"]
data = dict(label=["With URL  \r\n" + str(round(withUrl / totalData * 100, 2)) + "%"
                   , "Without URL  " + str(round(withoutUrl / totalData * 100, 2)) + "%"]
            , value=[withUrl, withoutUrl])

reset_output()
output_notebook()
show(Donut(data, label='label', values='value'))
# show(Bar(data, label="month", values="tweets_per_month", legend=False, color="black", plot_width=800),
#      notebook_handle=True)
# print ( json.dumps(response, indent = 2) )
# print (response["aggregations"]["users"]["value"])

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.010s]
INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.019s]


Loading BokehJS ...

### f. 請依照每個月繪製各Twitter所提到CVE的箱型圖(每月最多發表帳號的次數、每月平均發表的次數、每月至少發表的次數、第一分位數、第三分位數)

In [56]:
from elasticsearch import Elasticsearch
from bokeh.charts import BoxPlot, reset_output, output_notebook, show
from pandas import DataFrame
# from bokeh.io import output_notebook
import json

client = Elasticsearch('localhost:9200')

response = client.search(
    index = "twitter2",
    body = {
        "size" : 0,
        "_source": {
            "exclude": [ "user.*", "retweeted_status.*" ]
        },
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": {
                                "type": "boolean",
                                "query": "Reversing Vulnerability CVE exploitation Spoofing password DDoS SSL CryptoLocker Encrypt",
                                "operator": "or"
                            }
                        }
                    },
                    {
                        "range": {
                            "created_at": {
                                "gte": "Sun Jan 01 00:00:00 +0000 2012",
                                "lt": "Thu Jan 01 00:00:00 +0000 2015"
                            }
                        }
                    }
                ]
            }
        },
        "aggs": {
            "tweets_by_month": {
                "date_histogram": {
                    "field": "created_at",
                    "interval": "month",
                    "format": "yyyy-MM",
                    "min_doc_count" : 0
                },
                "aggs" : {
                    "user_posts_per_month" : {
                        "terms" : {
                            "field" : "uid"
                        }
                    }
                }
            }
        }
    }
)

data = dict(month=[], user_posts_per_month=[])
# print (json.dumps(response, indent = 2))
for d in response["aggregations"]["tweets_by_month"]["buckets"]:
    for count in d["user_posts_per_month"]["buckets"]:
        data["month"].append(d["key_as_string"])
        data["user_posts_per_month"].append(count["doc_count"])
# print (data)
dataframe = DataFrame.from_dict(data)
# print (DataFrame.from_dict(data))
p = BoxPlot(dataframe, values='user_posts_per_month', label='month', color="white", legend=False, plot_width=800)
reset_output()
output_notebook()
show(p, notebook_handle=True)
# print (json.dumps(response, indent = 2))
# print (response["aggregations"]["users"]["value"])

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.038s]


Loading BokehJS ...

### g.能否分析資安弱點指標帳號
=====
* 能

### g.如何分析
====
* 發文次數最多的人就是高手

In [57]:

from elasticsearch import Elasticsearch
import json

client = Elasticsearch('localhost:9200')

response = client.search(
    index = "twitter2",
    body = {
        "size" : 0,
        "_source": {
            "exclude": [ "user.*", "retweeted_status.*" ]
        },
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": {
                                "type": "boolean",
                                "query": "Reversing Vulnerability CVE exploitation Spoofing password DDoS SSL CryptoLocker Encrypt",
                                "operator": "or"
                            }
                        }
                    },
                    {
                        "range": {
                            "created_at": {
                                "gte": "Sun Jan 01 00:00:00 +0000 2012",
                                "lt": "Thu Jan 01 00:00:00 +0000 2015"
                            }
                        }
                    }
                ]
            }
        },
        "aggs": {
            "twitter_accounts_in_range": {
                "terms" : {
                    "size":1,
                    "field" : "uid"
                }
            }
        }
    }
)

# print (json.dumps(response, indent = 2))
print ("The expert uid is " + str(response["aggregations"]["twitter_accounts_in_range"]["buckets"][0]["key"]))
print ("They've posted " + str(response["aggregations"]["twitter_accounts_in_range"]["buckets"][0]["doc_count"]) + " tweets.")

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.021s]


The expert uid is 43130563
They've posted 236 tweets.
